In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.metrics as skmet

# Load the health information data

In [ ]:
df = pd.read_csv('/kaggle/input/spam-text-message-classification/SPAM text message 20170820 - Data.csv')

In [ ]:
df.head()

In [ ]:
df['spam'] = np.where(df.Category == 'spam', 1, 0)

In [ ]:
df.groupby(['spam', 'Category']).size()

# Create training and testing DataFrames

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[['Message']], df[['spam']], test_size=0.2,stratify=df[['spam']], random_state=0)

countvectorizer = CountVectorizer(analyzer='word',stop_words='english')

Using the vectorizer, we get counts for all non stop words used for each
observation. For example, like is used once in the first message and not at
all in the second. 

This gives like a value of 1 for the first observation in
the transformed data and a value of 0 for the second observation.
We won’t use anything from this step in our model. We are only doing this
for illustrative purposes:

In [ ]:
smallsample = X_train.loc[X_train.Message.str.len()<50].sample(2, random_state=35)
smallsample

In [ ]:
ourvec = \
  pd.DataFrame(countvectorizer.\
    fit_transform(smallsample.values.ravel()).toarray(),\
    columns=countvectorizer.get_feature_names_out())
ourvec

In [ ]:
#countvectorizer.fit(X_train.loc[X_train.message.str.len()<40].values.ravel())
nb = MultinomialNB()

smote = SMOTE(random_state=0)

pipe1 = make_pipeline(countvectorizer, smote, nb)

pipe1.fit(X_train.values.ravel(), y_train.values.ravel())

pred = pipe1.predict(X_test.values.ravel())

In [ ]:
print("accuracy: %.2f, sensitivity: %.2f, specificity: %.2f, precision: %.2f"  %
  (skmet.accuracy_score(y_test.values.ravel(), pred),
  skmet.recall_score(y_test.values.ravel(), pred),
  skmet.recall_score(y_test.values.ravel(), pred, pos_label=0),
  skmet.precision_score(y_test.values.ravel(), pred)))

In [ ]:
cm = skmet.confusion_matrix(y_test, pred)
cmplot = skmet.ConfusionMatrixDisplay(
confusion_matrix=cm, \
display_labels=['Not Spam', 'Spam'])
cmplot.plot()
cmplot.ax_.set(
title='Spam Prediction Confusion Matrix',
xlabel='Predicted Value', ylabel='Actual Value')

The metrics are often quite good and quite efficient.